## Notebook used to calculate all possible $\Phi$ values using modified PyPhi

In [4]:
import pyphi
import numpy as np
from pyphi import Direction
from pyphi import utils
from pyphi import cache

# Transition probability matrix. Saying where each state (little-endian binary) goes
tpm = np.array([
    [0.,0.,0.],
    [0.,0.,0.],
    [1.,0.,0.],
    [1.,0.,1.],
    [0.,1.,0.],
    [0.,1.,0.],
    [1.,1.,0.],
    [1.,1.,1.]
])

# Set up network object
network = pyphi.Network(tpm, node_labels=['R','C','P'])
print("NODE LABELS = ",network.node_labels)

# Put the system into a given state
state = (0,0,0)
nodes = ['R','C','P']
subsystem = pyphi.Subsystem(network, state, nodes)


mechanisms = utils.powerset(subsystem.node_indices, nonempty=True)
direction = Direction.CAUSE

print("MECHANISMS:")
for mechanism in mechanisms:
    print(mechanism)
    print("\tPotential Purviews:")
    potential_purviews = network.potential_purviews(direction, mechanism, purviews=False)
    for purview in potential_purviews:
        print("\t",purview)



# purviews = network.potential_purviews(Direction.CAUSE, mechanism,)

# Put the system into a given state
# state = (0,0,0)
# nodes = ['R','C','P']
# subsystem = pyphi.Subsystem(network, state, nodes)
# ces = pyphi.compute.ces(subsystem, mechanisms=False,purviews=False,cause_purviews=False,effect_purviews=False,parallel=False)
# print(ces.__dict__)
# # subsystem.concept

# # # sia = pyphi.compute.sia(subsystem)
# # # print("PHI = ",sia.phi)
# # print(sia.cut)

# ces = pyphi.compute.ces(subsystem, mechanisms=False,purviews=False,cause_purviews=False,effect_purviews=False,parallel=False)

# major_complex = pyphi.compute.major_complex(network, state)
# print(major_complex.subsystem)

# subsystem = major_complex.subsystem
# subsystem.concept()

NODE LABELS =  NodeLabels(('R', 'C', 'P'))
MECHANISMS:
(0,)
	Potential Purviews:


NotImplementedError: kwarg cache keys not implemented

In [1]:
from pyphi import utils

# subsystem.potential_purviews(self, 'CAUSE', mechanism, purviews=False):

        
for mechanism in utils.powerset('0','1','2'):
    print("MECHANISM = ",mechanism)
    print(subsystem.potential_purviews('CAUSE', mechanism, purviews=False))
#     for purview in utils.powerset('R','C'):
#         subsystem.cause_repertoire(mechanism,purview)

MECHANISM =  ('0',)


NameError: name 'subsystem' is not defined

#### Now get all the different cause/effect repetoires

In [8]:
# Put the system into a given state
state = (0,0,0)
nodes = ['R','C','P']
subsystem = pyphi.Subsystem(network, state, nodes)

# Get the cause repetoire for the mechanism over the purview
R,C,P = subsystem.node_indices
# print(subsystem.state)
mechanism = (C,)
purview = (R,P)
cr = subsystem.effect_repertoire(mechanism,purview)
flat_cr = pyphi.distribution.flatten(cr)  # returns 1d array in little endian convention
print(flat_cr)

mip = subsystem.effect_mip(mechanism,purview)
print(mip)

[1. 0. 0. 0.]
Repertoire irreducibility analysis
  φ = 1/4
  Mechanism: [C]
  Purview = [R, P]
  Direction: EFFECT
  Partition:
     ∅     C 
    ─── ✕ ───
     P     R 
  Repertoire:
    ┌──────────────┐
    │ S     Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
    │ 00    1      │
    │ 10    0      │
    │ 01    0      │
    │ 11    0      │
    └──────────────┘
  Partitioned repertoire:
    ┌──────────────┐
    │ S     Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
    │ 00    3/4    │
    │ 10    0      │
    │ 01    1/4    │
    │ 11    0      │
    └──────────────┘


## Now do the same for the unidirectional cut R -//->CP

In [9]:
import pyphi
import numpy as np

# Transition probability matrix. Saying where each state (little-endian binary) goes
tpm = np.array([
    [1.,0.,0.,0.,0.,0.,0.,0.],
    [1.,0.,0.,0.,0.,0.,0.,0.],
    [0.,1/2.,0.,0.,0.,1/2.,0.,0.],
    [0.,1/2.,0.,0.,0.,1/2.,0.,0.],
    [0.,0.,1.,0.,0.,0.,0.,0.],
    [0.,0.,1.,0.,0.,0.,0.,0.],
    [0.,0,0.,1/2.,0.,0.,0.,1/2.],
    [0.,0,0.,1/2.,0.,0.,0.,1/2.],
])

# Set up network object
# network = pyphi.Network(tpm, cm=cm, node_labels=['A','B','C'])
network = pyphi.Network(tpm, node_labels=['R','C','P'])

# Put the system into a given state
state = (0,0,0)
nodes = ['R','C','P']
subsystem = pyphi.Subsystem(network, state, nodes)

# Get the cause repetoire for the mechanism over the purview
R,C,P = subsystem.node_indices
# print(subsystem.state)
mechanism = (C,)
purview = (R,P)
cr = subsystem.effect_repertoire(mechanism,purview)
flat_cr = pyphi.distribution.flatten(cr)  # returns 1d array in little endian convention
print(flat_cr)

mip = subsystem.effect_mip(mechanism,purview)
print(mip)

[1. 0. 0. 0.]
Repertoire irreducibility analysis
  φ = 1/4
  Mechanism: [C]
  Purview = [R, P]
  Direction: EFFECT
  Partition:
     ∅     C 
    ─── ✕ ───
     P     R 
  Repertoire:
    ┌──────────────┐
    │ S     Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
    │ 00    1      │
    │ 10    0      │
    │ 01    0      │
    │ 11    0      │
    └──────────────┘
  Partitioned repertoire:
    ┌──────────────┐
    │ S     Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
    │ 00    3/4    │
    │ 10    0      │
    │ 01    1/4    │
    │ 11    0      │
    └──────────────┘


In [29]:
ps = pyphi.utils.powerset(['R','C','P'])
print(list(ps))
print(list(ps))
# for each in ps:
#     print(each)

[(), ('R',), ('C',), ('P',), ('R', 'C'), ('R', 'P'), ('C', 'P'), ('R', 'C', 'P')]
[]


In [22]:
list(ps)

[]